In [1]:
import os
from pathlib import Path
import shutil

import teehr

teehr.__version__

'0.6.0dev4'

In [2]:
from teehr.evaluation.spark_session_utils import create_spark_session

In [ ]:
spark = create_spark_session(
    start_spark_cluster=True,
    executor_instances=40,
    executor_memory="16g",
    executor_cores=4,
    aws_profile="default",
)

INFO:teehr.evaluation.spark_session_utils:🚀 Creating Spark session: TEEHR Evaluation
INFO:teehr.evaluation.spark_session_utils:📦 Configuring Spark cluster with container image: None
INFO:teehr.evaluation.spark_session_utils:🔍 Initial spark namespace from ENV: teehr-hub
INFO:teehr.evaluation.spark_session_utils:🔍 Connecting to Kubernetes API: https://172.20.0.1:443
INFO:teehr.evaluation.spark_session_utils:🎯 Executor namespace: teehr-hub
INFO:teehr.evaluation.spark_session_utils:🔐 Executor service account: spark (in teehr-hub)
INFO:teehr.evaluation.spark_session_utils:🔐 Using in-cluster authentication
INFO:teehr.evaluation.spark_session_utils:🔗 Setting driver host to pod IP: 10.0.2.254
INFO:teehr.evaluation.spark_session_utils:✅ Spark cluster configuration successful!
INFO:teehr.evaluation.spark_session_utils:   - Executor instances: 40
INFO:teehr.evaluation.spark_session_utils:   - Executor memory: 16g
INFO:teehr.evaluation.spark_session_utils:   - Executor cores: 4
INFO:teehr.evaluati

In [4]:
%%time
dir_path = "/data/temp_warehouse"

ev = teehr.Evaluation(
    spark=spark,
    dir_path=dir_path,
    check_evaluation_version=False,
)

INFO:teehr.evaluation.evaluation:Using provided Spark session.
INFO:teehr.evaluation.evaluation:Active catalog set to local.


CPU times: user 4.96 ms, sys: 59 μs, total: 5.02 ms
Wall time: 96.8 ms


In [5]:
ev.set_active_catalog("remote")

ev.active_catalog

INFO:teehr.evaluation.evaluation:Active catalog set to remote.


RemoteCatalog(warehouse_dir='s3://dev-teehr-iceberg-warehouse/', catalog_name='iceberg', namespace_name='teehr', catalog_type='rest', catalog_uri='http://iceberg-rest:8181')

In [6]:
ev.configurations.to_sdf().show()

INFO:teehr.evaluation.tables.base_table:Loading files from iceberg.teehr.configurations.
INFO:teehr.evaluation.read:Reading files from iceberg.teehr.configurations.


+--------------------+---------+--------------------+
|                name|     type|         description|
+--------------------+---------+--------------------+
|   nwm30_short_range|secondary|NWM short range -...|
| nwm30_retrospective|secondary|NWM 3.0 Retrospec...|
|   usgs_observations|  primary|   USGS Observations|
|  nwm30_medium_range|secondary|NWM medium range ...|
|nrds_v22_cfenom_s...|secondary|POC version of Da...|
+--------------------+---------+--------------------+



In [7]:
sdf = ev.joined_timeseries._join(secondary_filters=["configuration_name = 'nwm30_retrospective'"])

INFO:teehr.evaluation.tables.base_table:Setting filter <class 'filter'>.
INFO:teehr.evaluation.tables.base_table:Loading files from iceberg.teehr.primary_timeseries.
INFO:teehr.evaluation.read:Reading files from iceberg.teehr.primary_timeseries.
INFO:teehr.evaluation.read:Reading files from iceberg.teehr.primary_timeseries.
INFO:teehr.evaluation.tables.base_table:Setting filter <class 'filter'>.
INFO:teehr.evaluation.tables.base_table:Loading files from iceberg.teehr.secondary_timeseries.
INFO:teehr.evaluation.read:Reading files from iceberg.teehr.secondary_timeseries.
INFO:teehr.evaluation.read:Reading files from iceberg.teehr.secondary_timeseries.
INFO:teehr.evaluation.tables.base_table:Loading files from iceberg.teehr.secondary_timeseries.
INFO:teehr.evaluation.read:Reading files from iceberg.teehr.secondary_timeseries.
INFO:teehr.evaluation.tables.base_table:Loading files from iceberg.teehr.location_crosswalks.
INFO:teehr.evaluation.read:Reading files from iceberg.teehr.location_cr

In [8]:
sdf.show()

+--------------+-------------------+-------------------+---------------------+-------------+---------------+-------------------+---------+--------------------+------+
|reference_time|         value_time|primary_location_id|secondary_location_id|primary_value|secondary_value| configuration_name|unit_name|       variable_name|member|
+--------------+-------------------+-------------------+---------------------+-------------+---------------+-------------------+---------+--------------------+------+
|          NULL|1990-10-17 13:00:00|      usgs-01010000|        nwm30-4289887|     148.0971|         150.87|nwm30_retrospective|    m^3/s|streamflow_hourly...|  NULL|
|          NULL|1990-10-17 20:00:00|      usgs-01010000|        nwm30-4289887|    137.90305|         132.64|nwm30_retrospective|    m^3/s|streamflow_hourly...|  NULL|
|          NULL|1990-10-20 08:00:00|      usgs-01010000|        nwm30-4289887|    128.84164|          57.51|nwm30_retrospective|    m^3/s|streamflow_hourly...|  NULL

In [9]:
ev.write.to_warehouse(
        source_data=sdf,
        table_name="sim_joined_timeseries",
        write_mode="create_or_replace"
    )

In [10]:
spark.stop()